# Week 2. Web scraping

## This week's objectives

1. Understand how to scrape web pages and other data where an API doesn't exist
2. Introduce the `BeautifulSoup` library
3. Learn how to parse unstructured text data
4. Learn how to handle errors ("exceptions") gracefull
5. More pratice with `pandas`, `geopandas`, and plotting

APIs make it relatively simple to get data from the web. But sometimes, an API doesn't exist—they take effort on the part of the agency to set up and maintain.

In these cases, we can still obtain data from the web. But rather than dropping it directly into a (geo)pandas `DataFrame`, we'll need to do more work to understand the structure of the webpage, and to clean and process the results. 

## Example: Land use permit data
Often, cities make their building and land use permit data available for download, and/or accessible through an API. But these are typically incomplete—they provide a subset of fields that are most relevant to most users (e.g., permit approval date and number of units), but perhaps exclude more esoteric fields. And parking, sadly, is one of the fields that is often excluded.

For a recent project, I looked at the impacts of TOD plans in Seattle and San Francisco on development outcomes, including parking ratios. Let's walk through the Seattle analysis.

The [Seattle land use permit dataset is here](https://data.seattle.gov/Permitting/Land-Use-Permits/ht3q-kdvx). Let's get this into a `pandas` dataframe, in the same way that we did with the LA data.

In [1]:
%config Completer.use_jedi = False
import json
import requests
import pandas as pd
url = 'https://data.seattle.gov/resource/ht3q-kdvx.json' # copied and pasted from the webpage
r = requests.get(url)
df = pd.DataFrame(json.loads(r.text))
print(df.head())

    permitnum           permitclass permitclassmapped       permittypemapped  \
0  3009387-LU           Multifamily       Residential      Master Use Permit   
1  3020870-EG           Multifamily       Residential  Early Design Guidance   
2  3037371-LU           Multifamily       Residential      Master Use Permit   
3  3018857-LU            Commercial   Non-Residential      Master Use Permit   
4  3022144-LU  Single Family/Duplex       Residential      Master Use Permit   

                                         description applieddate expiresdate  \
0  Land use application to adjust the boundary be...  2008-07-25  2010-06-03   
1  Early Design Guidance for: Land use applicatio...         NaN         NaN   
2  Divide existing parcel into two equal resultan...         NaN         NaN   
3  Streamlined Design Review for a four story bui...         NaN         NaN   
4  Land Use Application to subdivide one developm...  2016-01-11  2019-05-10   

  decisiondate              statuscurr

Notice a couple of things. First, the website seems to have more fields than are in the API version of the document. Second, parking is nowhere to be seen.

But there is a `link` field. Let's take a look at the first one. 

In [2]:
df.columns

Index(['permitnum', 'permitclass', 'permitclassmapped', 'permittypemapped',
       'description', 'applieddate', 'expiresdate', 'decisiondate',
       'statuscurrent', 'originaladdress1', 'originalcity', 'originalstate',
       'originalzip', 'link', 'latitude', 'longitude', 'location1',
       'permittypedesc', 'estprojectcost', 'contractorcompanyname',
       'issueddate', 'housingunitsremoved', 'housingunitsadded',
       'housingunits'],
      dtype='object')

In [3]:
# The .loc operator gives us an extract from the dataframe. 0 is the row index, 'link' is the column

print(df.loc[0,'link'])   

{'url': 'https://cosaccela.seattle.gov/portal/customize/LinkToRecord.aspx?altId=3009387-LU'}


Notice that this column of the pandas dataframe is a dictionary. That's perhaps a surprise, but we know how to deal with dictionaries. 

For now, [let's take a look at what this link looks like](https://cosaccela.seattle.gov/portal/customize/LinkToRecord.aspx?altId=3003094-LU). Clearly, there is a lot more information here about the specific permit!

How do we bring the information in that webpage into Python? Remember, the `requests` library is our friend in this circumstance. While we've used it to get data from an API, `requests` can retrieve pretty much anything from the web.

In [4]:
urldict = df.loc[0,'link']
permiturl = urldict['url']
# or we could do this in one step: df.loc[0,'link']['url']
r = requests.get(permiturl)

In [10]:
# Let's look at what the r object has given us. 
# Remember, the .text attribute gives us the text of what's retrieved.
print(r.text)



<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html ng-app="appAca" xmlns="http://www.w3.org/1999/xhtml" lang="en-US" xml:lang="en-US" xmlns:og="http://ogp.me/ns#" xmlns:fb="http://www.facebook.com/2008/fbml">
<head id="ctl00_Head1"><link href="../App_Themes/Default/_progressbar.css" type="text/css" rel="stylesheet" /><link href="../App_Themes/Default/breadcrumb.css" type="text/css" rel="stylesheet" /><link href="../App_Themes/Default/Calendar.css" type="text/css" rel="stylesheet" /><link href="../App_Themes/Default/custom.css" type="text/css" rel="stylesheet" /><link href="../App_Themes/Default/font.css" type="text/css" rel="stylesheet" /><link href="../App_Themes/Default/form.css" type="text/css" rel="stylesheet" /><link href="../App_Themes/Default/grid.css" type="text/css" rel="stylesheet" /><link href="../App_Themes/Default/layout.css" type="text/css" rel="stylesheet" /><link href="../App_Themes/Default/n

It looks like we've got the whole .html webpage. The relevant information is buried in there, but how can we get it in the sea of html code?

This is where the `BeautifulSoup` library comes in ([documentation here](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)). Let's convert our text to a "soup" object.

In [11]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text)
print(type(soup))

<class 'bs4.BeautifulSoup'>


This soup object has a lot of attributes and functions (type `soup.` and press tab to autocomplete). We'll use the `.find` function to find the relevant text.

We can also use the `.prettify()` function to give us a better sense of what we are looking at.

In [12]:
# Not very pretty IMHO, but we can look at see where the data we want are buried
# and cross-refernce that to the webpage in our browser
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="en-US" ng-app="appAca" xml:lang="en-US" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head id="ctl00_Head1">
  <link href="../App_Themes/Default/_progressbar.css" rel="stylesheet" type="text/css"/>
  <link href="../App_Themes/Default/breadcrumb.css" rel="stylesheet" type="text/css"/>
  <link href="../App_Themes/Default/Calendar.css" rel="stylesheet" type="text/css"/>
  <link href="../App_Themes/Default/custom.css" rel="stylesheet" type="text/css"/>
  <link href="../App_Themes/Default/font.css" rel="stylesheet" type="text/css"/>
  <link href="../App_Themes/Default/form.css" rel="stylesheet" type="text/css"/>
  <link href="../App_Themes/Default/grid.css" rel="stylesheet" type="text/css"/>
  <link href="../App_Themes/Default/layout.css" rel="stylesheet" type="text/css"/>
  <link href="../Ap

Let's suppose we want to get information the project description (where the parking information might be included, since there isn't a separate parking field). 

It looks like this field are contained within a `<td>` tag. 

In [13]:
links = soup.find_all('td') # returns a "list-like" object, i.e. we can loop through it or slice it like a list

# Look at an example
print(links[5])

# More systematically, let's look at the links that we are interested
for link in links:
    if 'Project Description' in link.text: 
        print (link) 
        break # so that we keep this link, and abort the loop

<td class="ACA_TabRow_Line">
<a href="/Portal/Account/RegisterDisclaimer.aspx" id="ctl00_HeaderNavigation_btnRegister">
<span class="ACA_Body_Text ACA_Body_Text_FontSize" id="ctl00_HeaderNavigation_lblRegister"><i aria-hidden="true" class="material-icons">perm_identity</i> Register for an Account</span>
</a>
</td>
<td class="td_parent_left"><div>
<h1 style="font-size:1.4em;"><span id="ctl00_PlaceHolderMain_PermitDetailList1_per_permitDetail_label_projectl637534721557024130">Project Description</span></h1><span class="ACA_SmLabel ACA_SmLabel_FontSize"><table class="table_child" role="presentation" style="TEMPLATE_STYLE"><tr><td class="td_child_left font12px"></td><td>Land use application to adjust the boundary between two parcels of land. Proposed parcel sizes are:A)54,540 sq.ft.; B)21,169 sq.ft. 
Existing structures to be demolished.

</td></tr></table></span>
</div></td>


Now we are getting closer! It looks like the Project Description is contained in another `<td>` tag, nested one level down. So let's do the same thing again at this second-level link.

In [14]:
sublinks = link.find_all('td')
print(sublinks)

[<td class="td_child_left font12px"></td>, <td>Land use application to adjust the boundary between two parcels of land. Proposed parcel sizes are:A)54,540 sq.ft.; B)21,169 sq.ft. 
Existing structures to be demolished.

</td>]


We've obtained a list! And the information we need is in the second element of that list.

In [15]:
description = sublinks[1]
print(description.text)

Land use application to adjust the boundary between two parcels of land. Proposed parcel sizes are:A)54,540 sq.ft.; B)21,169 sq.ft. 
Existing structures to be demolished.




Now, let's take everything we've done so far, and put it in a function.
 
The function takes a single argument: the dictionary in the `url` column of the pandas DataFrame
 
It returns the Description text, unless that's not found, in which case it returns an empty string `''`.  

In [16]:
def getDescription(urldict):
    permiturl = urldict['url']
    # or we could do this in one step: df.loc[0,'link']['url']
    r = requests.get(permiturl)
    soup = BeautifulSoup(r.text)
    links = soup.find_all('td')
    for link in links:
        if 'Project Description' in link.text: 
            sublinks = link.find_all('td')
            description = sublinks[1].text
            # once we find a description, we return it and exit the function
            return description 
    
    return '' # if we don't find it, return an empty string

urldict = df.loc[0,'link']
getDescription(urldict)

'Land use application to adjust the boundary between two parcels of land. Proposed parcel sizes are:A)54,540 sq.ft.; B)21,169 sq.ft. \r\nExisting structures to be demolished.\r\n\r\n'

The advantage of a function is that we can now apply this procedure to every row of our pandas DataFrame.

Let's do this for 5 rows (so we are nice and don't disrupt the City's website).

The `apply` function in `pandas` applies a function to each row of a DataFrame.

In [18]:
smalldf = df.iloc[:5].copy()  # create a copy, rather than a view to that object
descriptions = smalldf['link'].apply(getDescription)  # for each row in smallDf, we pass the link column to getDescription

In [19]:
# what's the description object? It's a pandas Series (basically, a one-column DataFrame)
print(type(descriptions))
print(descriptions)

<class 'pandas.core.series.Series'>
0    Land use application to adjust the boundary be...
1    Early Design Guidance for: Land use applicatio...
2    Fauntleroy 7141 Lot C                Divide ex...
3    Streamlined Design Review for a four story bui...
4    Land Use Application to subdivide one developm...
Name: link, dtype: object


In [20]:
# So we can insert that into the dataframe as a new column
smalldf['description'] = descriptions
# we could have done this in one step: 
# smalldf['description'] = smalldf['link'].apply(getDescription) 

In [21]:
descriptions.values

array(['Land use application to adjust the boundary between two parcels of land. Proposed parcel sizes are:A)54,540 sq.ft.; B)21,169 sq.ft. \r\nExisting structures to be demolished.\r\n\r\n',
       'Early Design Guidance for: Land use application for streamlined design review for 4 three- story buildings - two residential units in each building (total of eight residential units). Surface parking for eight vehicles to be provided. Existing single family residence to be demolished.',
       'Fauntleroy 7141 Lot C                Divide existing parcel into two equal resultant lots via unit lot subdivision. Each resultant lot to include one dwelling unit attached by common wall at property line. Establish two legal parking spaces east of existing building.\n\nEstablish use as rowhouses and occupy, per plans.',
       'Streamlined Design Review for a four story building with two live work units and two residential units (two unit townhouse) with four parking spaces. Existing structure to b

Now we have scraped the description for each project!

How do we get the number of parking spaces? Well, that depends on whether the city uses consistent terminology. 

For starters, let's just get a column for whether there is "no parking" in the description.

In [26]:
import numpy as np

def noparking(description):
    if isinstance(description, str):
        text = description.lower()
        if 'no parking' in text:
            return True
        elif 'parking' in text:
            return False
    return np.nan
smalldf['noparking'] = smalldf.description.apply(noparking)

In [25]:
# example
isinstance('12345', str)

True

In [28]:
smalldf[['description', 'noparking']]


,description,noparking
0,Land use application to adjust the boundary be...,NaN
1,Early Design Guidance for: Land use applicatio...,False
2,Fauntleroy 7141 Lot C Divide ex...,False
3,Streamlined Design Review for a four story bui...,False
4,Land Use Application to subdivide one developm...,NaN


<div class="alert alert-block alert-info">
<strong>Exercise:</strong> If you want to get the number of parking spaces for each project, what would be your next step? In principle, how might you do that?
</div>

<div class="alert alert-block alert-info">
<strong>Let's generalize.</strong> What did we do here?
    
1. We obtained the URL for each page to scrape. (Here, it was given to us in the city's data file, but sometimes we'll have to reverse-engineer the composition of the URL.)
2. We examined a sample page, and identified the html tags that enclose the data we wanted to extract.
3. We wrote a function that pulled out the data for a specific page.
4. We applied that function to each URL / page. Since our URLs were in a pandas DataFrame, we could use the pandas <strong>apply</strong> method.
    
Every scraping project will pose different challenges, but normally it will involve each of these four steps.
</div>